# 巡回セールスマン問題

## 問題の定式化
$$ Minimize \sum_{ij} c_{ij}x{ij} $$
$$ start \sum_j x_{ij} = 1 $$
$$ \sum_{j} x_{ij} = 1 $$
$$ u_i + 1 - BigM(1-x_{ij}) \le u_j $$
$$ 1 \le u_i \le n-1 $$
$$ x_{ij} \in \{0, 1\} $$

- $x_{ij}$ : 拠点 $i$ から拠点 $j$ に移動する場合は $1$ 、しない場合は $0$
- $c_{ij}$ : 拠点 $i$ から拠点 $j$ の移動距離を示す定数
- 
- 

# 旅行ルート最小化問題

## パラメータ
- $ V = \{0, 1, 2, 3, 4, 5 \} $ : 都市の集合
- $ d_{ij} $ : 都市間距離
## 変数
- $ x_{ij} \in \{0, 1\} $ : 都市の移動するしないの0-1変数
- $ y_i $ : MTX 制約で用いる連続変数
## 目的関数
$$ minimize \sum_{i\in V} \sum_{j\in V, i\neq j} d_{ij} x_{ij} $$
## 制約条件
$$ \sum_{j\in V, i\neq j} x_{ij} = 1 \quad (\Lambda_i \in V) $$
$$ \sum_{i\in V, i\neq j} x_{ij} = 1 \quad (\Lambda_j \in V) $$
$$ y_i - y_j + |V|x_{ij} \le |V| - 1 \quad (\Lambda_i \in V / \{0\}, \Lambda_j \in V / \{0\}) $$


In [127]:
from pulp import LpProblem, LpVariable, LpStatus, LpMinimize, lpSum, value
import pandas as pd

In [128]:
## 記号の設定
V = [0, 1, 2, 3, 4, 5]
V_0 = [1, 2, 3, 4, 5]

# # 各都市間の距離を設定
d = {}
d[0, 1] = 10862
d[0, 2] = 18388
d[0, 3] = 1152
d[0, 4] = 9570
d[0, 5] = 9575
d[1, 2] = 8536
d[1, 3] = 11070
d[1, 4] = 5577
d[1, 5] = 9030
d[2, 3] = 19454
d[2, 4] = 11142
d[2, 5] = 11829
d[3, 4] = 8873
d[3, 5] = 8498
d[4, 5] = 3513

# 距離dを対称にする
for i in V:
    for j in V:
        if i > j:
            d[i, j] = d[j, i]


# 問題の定式化
m = LpProblem(sense=LpMinimize)

x = LpVariable.dicts('x', [(i, j) for i in V for j in V if i != j], cat='Binary')
y = LpVariable.dicts('y', V_0, cat='Continuous')

m += lpSum(d[(i, j)] * x[i, j] for i in V for j in V if i != j)

for i in V:
    m += lpSum(x[i, j] for j in V if i != j) == 1

for j in V:
    m += lpSum(x[i, j] for i in V if i != j) == 1

for i in V_0:
    for j in V_0:
        if i != j:
            m += y[i] - y[j] + len(V) * x[i, j] <= len(V) - 1

result = m.solve()
print(LpStatus[result])

# 結果表示
for i in V:
    for j in V:
        if i != j:
            if x[i, j].value() == 1:
                print(f'x_{i}_{j} = ', x[i, j].value())

print(f'最適値 : {m.objective.value()}')

Optimal
x_0_1 =  1.0
x_1_2 =  1.0
x_2_4 =  1.0
x_3_0 =  1.0
x_4_5 =  1.0
x_5_3 =  1.0
最適値 : 43703.0


In [133]:
D = [
    [0, 1, 10862],
    [0, 2, 18388],
    [0, 3, 1152],
    [0, 4, 9570],
    [0, 5, 9575],
    [1, 2, 8536],
    [1, 3, 11070],
    [1, 4, 5577],
    [1, 5, 9030],
    [2, 3, 19454],
    [2, 4, 11142],
    [2, 5, 11829],
    [3, 4, 8873],
    [3, 5, 8498],
    [4, 5, 3513],
]
D_opp = [[d[1], d[0], d[2]] for d in D]

In [143]:
# 問題の定式化
m = LpProblem(sense=LpMinimize)

df = pd.DataFrame(D + D_opp, columns=['i', 'j', 'distance'])
df['x'] = [LpVariable(f'x_{r.i}_{r.j}', cat='Binary') for r in df.itertuples()]
y = LpVariable.dicts('y', V_0, cat='Continuous')

m += lpSum(df['x'] * df['distance'])

for k, v in df.groupby('i'):
    # print(v['x'])
    m += lpSum(v['x']) == 1

for k, v in df.groupby('j'):
    # print(v['x'])
    m += lpSum(v['x']) == 1

for i in V_0:
    for j in V_0:
        if i != j:
            m += y[i] - y[j] + len(V) * df[(df['i'] == i) & (df['j'] == j)]['x'] <= len(V) - 1


In [144]:
result = m.solve()
print(LpStatus[result])

df['result'] = df['x'].apply(value)
result_df = df[df['result'] == 1]
print(f'最適値 : {m.objective.value()}')
result_df

Optimal
最適値 : 43703.0


,i,j,distance,x,result
0,0,1,10862,x_0_1,1.0
5,1,2,8536,x_1_2,1.0
10,2,4,11142,x_2_4,1.0
14,4,5,3513,x_4_5,1.0
17,3,0,1152,x_3_0,1.0
28,5,3,8498,x_5_3,1.0
